In [ ]:
#### Goal 1
Convert UniTime XML into consistent JSONL dataset for supervised fine tuning


SyntaxError: invalid syntax (ipython-input-1542275232.py, line 2)

In [ ]:
# Cell 0: Imports
# Purpose: Import core libraries needed for XML parsing, data handling, and file writing.

import xml.etree.ElementTree as ET
import pandas as pd
import json


In [ ]:
# Cell 1: Load XML file
# Purpose: Read the UniTime XML into an ElementTree object for traversal.

xml_path = "/content/pu-spr07-llr.xml"
tree = ET.parse(xml_path)
root = tree.getroot()


In [ ]:
# Cell 2: Extract rooms
# Purpose: Create a DataFrame with room_id, capacity, and location.

rooms = []
for r in root.find("rooms"):
    rooms.append({
        "room_id": int(r.get("id")),
        "capacity": int(r.get("capacity")),
        "location": r.get("location")
    })

df_rooms = pd.DataFrame(rooms)
df_rooms.head()


,room_id,capacity,location
0,1,75,"450,406"
1,2,84,"410,398"
2,3,118,"451,435"
3,4,82,"451,435"
4,5,70,"487,447"


In [ ]:
# Cell 3: Extract instructors
# Purpose: Create a DataFrame with instructor_id and attributes.

instructors = []
for inst in root.find("instructors"):
    instructors.append({"instructor_id": int(inst.get("id"))})

df_instructors = pd.DataFrame(instructors)
df_instructors.head()


,instructor_id
0,163
1,197
2,273
3,305
4,307


In [ ]:
# Cell 0: Imports & helpers (robust)
import xml.etree.ElementTree as ET
import pandas as pd
import json
import logging
logging.basicConfig(level=logging.INFO, format="%(levelname)s: %(message)s")

def to_int(x, default=None):
    try:
        return int(x)
    except (TypeError, ValueError):
        return default

def to_float(x, default=None):
    try:
        return float(x)
    except (TypeError, ValueError):
        return default



In [ ]:
# Cell 4: Extract classes (robust to missing attributes)
classes = []
classes_tag = root.find("classes")
if classes_tag is None:
    raise RuntimeError("No <classes> section found in XML")

for c in classes_tag:
    rec = {
        "class_id": to_int(c.get("id")),
        "offering": to_int(c.get("offering")),
        "config": to_int(c.get("config")),          # may be None
        "subpart": to_int(c.get("subpart")),        # may be None
        "class_limit": to_int(c.get("classLimit")), # may be None
        "min_limit": to_int(c.get("minLimit")),     # optional
        "max_limit": to_int(c.get("maxLimit")),     # optional
        "dates_mask": c.get("dates"),
        "nr_rooms": to_int(c.get("nrRooms")),
        "scheduler_dept": to_int(c.get("scheduler")),
        "committed": c.get("committed") in ("true","True","1")
    }
    classes.append(rec)

df_classes = pd.DataFrame(classes)
logging.info("Parsed %d classes", len(df_classes))
logging.info("Missing values:\n%s", df_classes.isna().sum())
df_classes.head()


,class_id,offering,config,subpart,class_limit,min_limit,max_limit,dates_mask,nr_rooms,scheduler_dept,committed
0,1,1,1,1,66.0,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
1,2,2,2,2,90.0,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
2,3,3,3,3,80.0,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
3,4,4,4,4,60.0,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
4,5,5,5,5,54.0,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False


In [ ]:
# Cell 4.1: Show classes missing class_limit/config/subpart
missing = df_classes[df_classes[["class_limit","config","subpart"]].isna().any(axis=1)]
logging.info("Classes missing key fields: %d", len(missing))
missing.head(20)


,class_id,offering,config,subpart,class_limit,min_limit,max_limit,dates_mask,nr_rooms,scheduler_dept,committed
87,88,72,72,77,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
88,89,72,72,77,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
89,90,72,72,78,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
90,91,72,72,78,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
91,92,72,72,78,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
272,273,224,224,241,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
351,352,292,292,311,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
352,353,292,292,311,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
353,354,293,293,312,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False
354,355,293,293,312,NaN,None,None,0000000000000000000000000000000000000011111100...,NaN,2,False


In [ ]:
# Cell 5: Candidate rooms (robust)
class_room_candidates = []
for c in root.find("classes"):
    cid = to_int(c.get("id"))
    for r in c.findall("room"):
        rid = to_int(r.get("id"))
        pref = to_float(r.get("pref"), 0.0)
        if cid is None or rid is None:
            logging.warning("Skip room candidate with missing ids: class=%s room=%s", cid, rid)
            continue
        class_room_candidates.append({"class_id": cid, "room_id": rid, "pref": pref})

df_class_rooms = pd.DataFrame(class_room_candidates)
logging.info("Class-room candidates: %d", len(df_class_rooms))
df_class_rooms.head()


,class_id,room_id,pref
0,1,1,4.0
1,1,2,4.0
2,1,3,0.0
3,1,4,0.0
4,1,5,0.0


In [ ]:
# Cell 6: Candidate times (robust)
class_time_candidates = []
for c in root.find("classes"):
    cid = to_int(c.get("id"))
    for t in c.findall("time"):
        days = t.get("days")
        start = to_int(t.get("start"))
        length = to_int(t.get("length"))
        pref = to_float(t.get("pref"), 0.0)
        if cid is None or days is None or start is None or length is None:
            logging.warning("Skip time candidate with missing fields for class %s", cid)
            continue
        class_time_candidates.append({"class_id": cid, "days": days, "start": start, "length": length, "pref": pref})

df_class_times = pd.DataFrame(class_time_candidates)
logging.info("Class-time candidates: %d", len(df_class_times))
df_class_times.head()


,class_id,days,start,length,pref
0,1,1010100,90,12,2.0
1,1,1010100,102,12,0.0
2,1,1010100,114,12,0.0
3,1,1010100,126,12,0.0
4,1,1010100,138,12,-10.0


In [ ]:
# Cell 7: Class instructors (robust)
class_instructors = []
for c in root.find("classes"):
    cid = to_int(c.get("id"))
    for inst in c.findall("instructor"):
        iid = to_int(inst.get("id"))
        if cid is None or iid is None:
            logging.warning("Skip instructor link with missing ids: class=%s inst=%s", cid, iid)
            continue
        class_instructors.append({"class_id": cid, "instructor_id": iid})

df_class_instructors = pd.DataFrame(class_instructors)
logging.info("Class-instructor links: %d", len(df_class_instructors))
df_class_instructors.head()


,class_id,instructor_id
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [ ]:
# Cell 8: Build JSONL for per-class training (robust)
jsonl_path = "timetable_per_class.jsonl"
jsonl_records = []
nr_days = to_int(root.get("nrDays"), 7)
slots_per_day = to_int(root.get("slotsPerDay"), 288)

for _, row in df_classes.iterrows():
    cid = row["class_id"]
    cand_times = df_class_times[df_class_times["class_id"] == cid][["days","start","length","pref"]].to_dict("records")
    cand_rooms = df_class_rooms[df_class_rooms["class_id"] == cid][["room_id","pref"]].rename(columns={"room_id":"id"}).to_dict("records")
    insts = df_class_instructors[df_class_instructors["class_id"] == cid]["instructor_id"].tolist()

    if not cand_times or not cand_rooms:
        continue  # skip unlearnable examples

    jsonl_records.append({
        "instruction": "Assign a feasible room and time for the class given the candidates and constraints.",
        "input": {
            "nr_days": nr_days,
            "slots_per_day": slots_per_day,
            "classes": [{
                "class_id": cid,
                "class_limit": row.get("class_limit"),
                "dates_mask": row.get("dates_mask"),
                "instructors": insts,
                "candidate_times": cand_times,
                "candidate_rooms": cand_rooms
            }]
        },
        "output": {"assignments": []}
    })

with open(jsonl_path, "w", encoding="utf-8") as f:
    for r in jsonl_records:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

logging.info("Wrote %d JSONL records → %s", len(jsonl_records), jsonl_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cell 10: Gold assignment extractor (with fallback)
# Purpose: Populate a table of gold placements {class_id, days, start, length, room_id}.

gold_rows = []

# 10A) Look for explicit assignments blocks (common in some UniTime dumps)
assign_root = root.find("assignments") or root.find("solutions") or root.find("solution")
if assign_root is not None:
    # Example formats vary; we try a couple of common patterns
    for a in assign_root.findall(".//assignment"):
        cid = to_int(a.get("classId")) or to_int(a.get("class"))
        days = a.get("days")
        start = to_int(a.get("start"))
        length = to_int(a.get("length"))
        # room may be nested or as attr; handle both styles
        rid = to_int(a.get("room"))  # attr style
        if rid is None:
            room_tag = a.find("room")
            rid = to_int(room_tag.get("id")) if room_tag is not None else None
        if cid and days and start is not None and length is not None and rid is not None:
            gold_rows.append({"class_id": cid, "days": days, "start": start, "length": length, "room_id": rid})

# 10B) If no explicit gold found, synthesize pseudo-gold from candidates
if not gold_rows:
    # Capacity lookup
    room_cap = {}
    for r in root.find("rooms"):
        rid = to_int(r.get("id"))
        cap = to_int(r.get("capacity"))
        if rid is not None: room_cap[rid] = cap

    for _, row in df_classes.iterrows():
        cid = row["class_id"]
        limit_ = row.get("class_limit") or 0

        cand_times = df_class_times[df_class_times.class_id == cid].copy()
        cand_rooms = df_class_rooms[df_class_rooms.class_id == cid].copy()

        if cand_times.empty or cand_rooms.empty:
            continue

        # Filter rooms by capacity if we have it
        cand_rooms["capacity_ok"] = cand_rooms["room_id"].map(lambda rid: (room_cap.get(rid, 10**9) >= limit_))
        feasible_rooms = cand_rooms[cand_rooms["capacity_ok"]]

        if feasible_rooms.empty:
            feasible_rooms = cand_rooms  # fallback: no capacity info -> allow

        # Best (highest) pref; UniTime uses higher "pref" as better in your sample
        best_room = feasible_rooms.sort_values("pref", ascending=False).head(1).iloc[0]
        best_time = cand_times.sort_values("pref", ascending=False).head(1).iloc[0]

        gold_rows.append({
            "class_id": cid,
            "days": best_time["days"],
            "start": int(best_time["start"]),
            "length": int(best_time["length"]),
            "room_id": int(best_room["room_id"])
        })

df_gold = pd.DataFrame(gold_rows).drop_duplicates(subset=["class_id"])
len(df_gold), df_gold.head()


(803,
    class_id     days  start  length  room_id
 0         1  1010100     90      12        1
 1         2  1010100     90      12       16
 2         3  1010100     90      12        2
 3         4  1010100     90      12        1
 4         5  1010100     90      12        1)

In [ ]:
# Cell 11: Per-class JSONL (with gold assignments)
out_path = "timetable_per_class.jsonl"
nr_days = to_int(root.get("nrDays"), 7)
slots_per_day = to_int(root.get("slotsPerDay"), 288)

wrote = 0
with open(out_path, "w", encoding="utf-8") as f:
    for _, row in df_classes.iterrows():
        cid = row["class_id"]
        cand_times = df_class_times[df_class_times.class_id == cid][["days","start","length","pref"]].to_dict("records")
        cand_rooms = df_class_rooms[df_class_rooms.class_id == cid][["room_id","pref"]].rename(columns={"room_id":"id"}).to_dict("records")
        insts = df_class_instructors[df_class_instructors.class_id == cid]["instructor_id"].tolist()
        gold = df_gold[df_gold.class_id == cid]
        if not cand_times or not cand_rooms or gold.empty:
            continue

        g = gold.iloc[0]
        record = {
            "instruction": "Assign a feasible room and time for the class given the candidates and constraints.",
            "input": {
                "nr_days": nr_days,
                "slots_per_day": slots_per_day,
                "classes": [{
                    "class_id": int(cid),
                    "class_limit": row.get("class_limit"),
                    "dates_mask": row.get("dates_mask"),
                    "instructors": list(map(int, insts)) if insts else [],
                    "candidate_times": cand_times,
                    "candidate_rooms": cand_rooms
                }]
            },
            "output": {
                "assignments": [{
                    "class_id": int(cid),
                    "room": int(g["room_id"]),
                    "time": {"days": g["days"], "start": int(g["start"]), "length": int(g["length"])}
                }]
            }
        }
        f.write(json.dumps(record, ensure_ascii=False) + "\n")
        wrote += 1

print(f"Wrote {wrote} per-class JSONL records to {out_path}")


Wrote 803 per-class JSONL records to timetable_per_class.jsonl


In [ ]:
# Cell 12: Per-offering JSONL
# Purpose: One sample per offering containing all its classes + gold assignments.

off_jsonl = "timetable_per_offering.jsonl"
wrote_off = 0

# Group classes by offering
classes_by_off = df_classes.groupby("offering")

with open(off_jsonl, "w", encoding="utf-8") as f:
    for off_id, grp in classes_by_off:
        classes_bundle = []
        gold_bundle = []

        for _, row in grp.iterrows():
            cid = row["class_id"]
            cand_times = df_class_times[df_class_times.class_id == cid][["days","start","length","pref"]].to_dict("records")
            cand_rooms = df_class_rooms[df_class_rooms.class_id == cid][["room_id","pref"]].rename(columns={"room_id":"id"}).to_dict("records")
            insts = df_class_instructors[df_class_instructors.class_id == cid]["instructor_id"].tolist()
            if not cand_times or not cand_rooms:   # skip empty candidates
                continue
            classes_bundle.append({
                "class_id": int(cid),
                "class_limit": row.get("class_limit"),
                "dates_mask": row.get("dates_mask"),
                "instructors": list(map(int, insts)) if insts else [],
                "candidate_times": cand_times,
                "candidate_rooms": cand_rooms
            })
            g = df_gold[df_gold.class_id == cid]
            if not g.empty:
                gg = g.iloc[0]
                gold_bundle.append({
                    "class_id": int(cid),
                    "room": int(gg["room_id"]),
                    "time": {"days": gg["days"], "start": int(gg["start"]), "length": int(gg["length"])}
                })

        if not classes_bundle or not gold_bundle:
            continue

        rec = {
            "instruction": f"Assign feasible rooms and times for all classes in offering {int(off_id)}.",
            "input": {
                "nr_days": nr_days,
                "slots_per_day": slots_per_day,
                "offering": int(off_id),
                "classes": classes_bundle
            },
            "output": {"assignments": gold_bundle}
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
        wrote_off += 1

print(f"Wrote {wrote_off} per-offering JSONL records to {off_jsonl}")



Wrote 603 per-offering JSONL records to timetable_per_offering.jsonl


In [ ]:
# Cell 13: Persist normalized tables
df_rooms.to_csv("rooms.csv", index=False)
df_classes.to_csv("classes.csv", index=False)
df_class_times.to_csv("class_candidate_time.csv", index=False)
df_class_rooms.to_csv("class_candidate_room.csv", index=False)
df_class_instructors.to_csv("class_instructor.csv", index=False)

# Optional Parquet
# for name, df in {"rooms":df_rooms, "classes":df_classes, "class_time":df_class_times,
#                  "class_room":df_class_rooms, "class_instr":df_class_instructors}.items():
#     df.to_parquet(f"{name}.parquet")
print("Saved CSVs.")


Saved CSVs.


In [ ]:
# Cell 14: ETL report
report = {
    "n_rooms": len(df_rooms),
    "n_classes": len(df_classes),
    "n_class_with_candidates": int(df_class_times.class_id.nunique() & df_class_rooms.class_id.nunique()),
    "n_gold": len(df_gold),
}
report


{'n_rooms': 55,
 'n_classes': 803,
 'n_class_with_candidates': 803,
 'n_gold': 803}

In [ ]:
# Cell 15: Per-offering JSONL (Task B)
# Purpose: One JSONL line per offering containing all its classes, candidates, and gold assignments.

off_jsonl_path = "timetable_per_offering.jsonl"
nr_days = to_int(root.get("nrDays"), 7)
slots_per_day = to_int(root.get("slotsPerDay"), 288)

# Keep only classes that have both candidates and gold
have_time = set(df_class_times.class_id.unique())
have_room = set(df_class_rooms.class_id.unique())
have_gold = set(df_gold.class_id.unique())
eligible_class_ids = have_time & have_room & have_gold

df_classes_elig = df_classes[df_classes.class_id.isin(eligible_class_ids)].copy()

# Some XMLs may miss "offering" on a few classes; drop those rows for Task B
df_classes_elig = df_classes_elig[~df_classes_elig["offering"].isna()].copy()
df_classes_elig["offering"] = df_classes_elig["offering"].astype(int)

written = 0
with open(off_jsonl_path, "w", encoding="utf-8") as f:
    for off_id, grp in df_classes_elig.groupby("offering"):
        classes_bundle = []
        gold_bundle = []

        for _, row in grp.iterrows():
            cid = int(row["class_id"])

            cand_times = (df_class_times[df_class_times.class_id == cid]
                          [["days","start","length","pref"]].to_dict("records"))
            cand_rooms = (df_class_rooms[df_class_rooms.class_id == cid]
                          [["room_id","pref"]]
                          .rename(columns={"room_id":"id"})
                          .to_dict("records"))
            insts = df_class_instructors[df_class_instructors.class_id == cid]["instructor_id"].tolist()

            g = df_gold[df_gold.class_id == cid].iloc[0]

            classes_bundle.append({
                "class_id": cid,
                "class_limit": row.get("class_limit"),
                "dates_mask": row.get("dates_mask"),
                "instructors": list(map(int, insts)) if insts else [],
                "candidate_times": cand_times,
                "candidate_rooms": cand_rooms
            })

            gold_bundle.append({
                "class_id": cid,
                "room": int(g["room_id"]),
                "time": {"days": g["days"], "start": int(g["start"]), "length": int(g["length"])}
            })

        if not classes_bundle or not gold_bundle:
            continue

        rec = {
            "instruction": f"Assign feasible rooms and times for all classes in offering {off_id}.",
            "input": {
                "nr_days": nr_days,
                "slots_per_day": slots_per_day,
                "offering": int(off_id),
                "classes": classes_bundle
            },
            "output": {"assignments": gold_bundle}
        }
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
        written += 1

print(f"Wrote {written} per-offering JSONL records → {off_jsonl_path}")


Wrote 603 per-offering JSONL records → timetable_per_offering.jsonl


In [ ]:
# Cell 16: Preview the first two per-offering records
with open("timetable_per_offering.jsonl", "r", encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 2: break
        print(line[:400] + ("..." if len(line) > 400 else ""))


{"instruction": "Assign feasible rooms and times for all classes in offering 1.", "input": {"nr_days": 7, "slots_per_day": 288, "offering": 1, "classes": [{"class_id": 1, "class_limit": 66.0, "dates_mask": "00000000000000000000000000000000000000111111001111101111110111111011111101111110111111011111101111110000000011111101111110111111011111101111110111111", "instructors": [1], "candidate_times": [{...
{"instruction": "Assign feasible rooms and times for all classes in offering 2.", "input": {"nr_days": 7, "slots_per_day": 288, "offering": 2, "classes": [{"class_id": 2, "class_limit": 90.0, "dates_mask": "00000000000000000000000000000000000000111111001111101111110111111011111101111110111111011111101111110000000011111101111110111111011111101111110111111", "instructors": [2], "candidate_times": [{...
